- Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
- Apply a distortion correction to raw images.
- Use color transforms, gradients, etc., to create a thresholded binary image.
- Apply a perspective transform to rectify binary image ("birds-eye view").
- Detect lane pixels and fit to find the lane boundary.
- Determine the curvature of the lane and vehicle position with respect to center.
- Warp the detected lane boundaries back onto the original image.
- Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
# Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
# todo(qingyouz): prototype here, then move to a calibration file
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np


In [9]:
# prepare object points
nx = 9
ny = 6

# Make a list of calibration images
camera_cal = 'camera_cal'
camera_cal_images = os.listdir(camera_cal)

# Hard code the points based on https://docs.opencv.org/3.4/dc/dbb/tutorial_py_calibration.html 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
def get_objp(nx, ny):
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
    return objp

check_objp = get_objp(nx, ny)
print(check_objp.shape)

# performs the camera calibration, image distortion correction and 
# returns the undistorted image
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()\
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)
    undist = np.copy(img)  # Delete this line
    undist = cv2.undistort(img, mtx, dist, None, None)
    return undist

objpoints= []
imgpoints= []

for fname in camera_cal_images:
    f = os.path.join(camera_cal, fname)
    print(f)
    img = cv2.imread(os.path.join(camera_cal, fname))
    print(img.shape)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    # If found, draw corners and append them to objpoints and imgpoints
    if ret == True:
        # Draw and display the corners
        # print(corners)
        objpoints.append(check_objp)
        imgpoints.append(corners)

        cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
        plt.imshow(img)

# Now we gathered all points calibrate
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


[[1.15777930e+03 0.00000000e+00 6.67111054e+02]
 [0.00000000e+00 1.15282291e+03 3.86128937e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[982.00366211   0.         679.60492147]
 [  0.         968.17901611 388.5261257 ]
 [  0.           0.           1.        ]]


In [15]:

img = cv2.imread(os.path.join(camera_cal, 'calibration1.jpg'))
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

print(mtx)
print(newcameramtx)


[[1.15777930e+03 0.00000000e+00 6.67111054e+02]
 [0.00000000e+00 1.15282291e+03 3.86128937e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[982.00366211   0.         679.60492147]
 [  0.         968.17901611 388.5261257 ]
 [  0.           0.           1.        ]]


In [20]:
# undistort and store
# todo(qingyouz): 
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite(os.path.join('output_images', 'calibresult.png'), dst)

True

In [32]:
# Apply distortion to raw images
raw_imgs = os.listdir('test_images/')
undistort_imgs = []
for i in raw_imgs:
    img = cv2.imread(os.path.join('test_images/', i))
    undistort = cv2.undistort(img, mtx, dist, None, None)
    undistorted_file = os.path.join('output_images', 'ud'+i)
    cv2.imwrite(undistorted_file, undistort)
    undistort_imgs.append(undistorted_file)

In [40]:
# Use color transforms, gradients, etc., to create a thresholded binary image
# Edit this function to create your own pipeline.
def thresholded_pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    return color_binary
    
color_binaries = []
for filename in undistort_imgs:
    img = cv2.imread(filename)
    color_binary = thresholded_pipeline(img)
    ext = os.path.splitext(filename)[-1]
    f = os.path.join(*os.path.splitext(filename)[:-1]) + "_color_binaries"+ext
    color_binaries.append(f)
    cv2.imwrite(f, color_binary)

